In [48]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
import gspread as gs
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting.dataframe import format_with_dataframe
from scipy.optimize import minimize
import yfinance as yf
import jupytab

def google_caller():
    gc = gs.service_account(filename='dva-api-384400-c063824c0207.json')
    gc
 
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nS8WRaVB5S2aaxMiFpVHxSrEE12lx6TGs_WyAkcUSzU/edit?usp=sharing')
    ws = sh.worksheet('Form Responses 1')
 
    df = pd.DataFrame(ws.get_all_records())
 
    df = df[['Timestamp', 'Risk Assessment Score']].rename(columns={'Timestamp':'time', 'Risk Assessment Score':'score'})
    df = df.sort_values(by='time', ascending=False).head(1)
    return df
 
 
df = google_caller()
def ticker_picker(df):
    if df['score'].iloc[0] <= 12:
        tickers = ['aapl','adbe','amd','csco', 'fisv', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'orcl', 'qcom', 'txn', 'nvda', 'fis', 'crm', 'avgo', 'now']
 
    elif (df['score'].iloc[0] > 12) & (df['score'].iloc[0] <= 20):
        tickers = ['csgp','mdu','amd','csco', 'lly', 'ibm', 'intc', 'wba', 'isrg', 'pep', 'orcl', 'qcom', 'aciw', 'nvda', 'qlys', 'crm', 'cost', 'now']
 
    elif (df['score'].iloc[0] > 20) & (df['score'].iloc[0] <= 28):
        tickers = ['aapl','adbe','amd','csco', 'fisv', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'orcl', 'nbix', 'txn', 'nvda', 'fis', 'crm', 'avgo', 'saic']
 
    elif (df['score'].iloc[0] > 29) & (df['score'].iloc[0] <= 34):
        tickers = ['dci','adbe','mrcy','wtrg', 'fisv', 'ibm', 'axs', 'lrcx', 'msft', 'bros', 'orcl', 'pypl', 'txn', 'amrc', 'fis', 'crm', 'chgg', 'now']
 
    elif (df['score'].iloc[0] > 35):
        tickers = ['meta','adbe','amd','csco', 'fisv', 'ibm', 'ale', 'lrcx', 'jazz', 'mu', 'orcl', 'qcom', 'txn', 'nvda', 'fis', 'crm', 'eqix', 'vrsk']
    
    stock_data = yf.download(tickers,start='2017-01-1', end='2018-12-1')
    stock_data = stock_data.dropna()
    stocks = stock_data['Adj Close']
    return stocks, tickers
ticker_picker(df)
 
 
google_df = google_caller()
stocks = ticker_picker(google_df)[0]
tickers = ticker_picker(google_df)[1]

def efficient_frontier_calculator(stocks):
    np.random.seed(100)
 
    log_ret = np.log(stocks/stocks.shift(1))
    log_ret.head()
 
    w_random = np.random.random(18)
    weights = np.array(w_random)
    # print(weights/np.sum(weights))
    # print('\n\n')
 
    weight_log = log_ret.mean() * weights
    expected_returns = np.sum(weight_log) *252
    # print(expected_returns)
    # print('\n')
 
    transformed_weight = weights.T
    day_scale = 252
    expected_volatility = np.sqrt(np.dot(transformed_weight, np.dot(log_ret.cov() * day_scale, weights)))
    # print(expected_volatility)
    # print('\n')
 
    sharpe_ratio = expected_returns/expected_volatility
    # print(sharpe_ratio)
    return log_ret, weights
log_ret, efc = efficient_frontier_calculator(stocks)

def weight_methodology(stocks, log_ret, weights):
    num_of_stocks = len(stocks.columns)
    all_weights = np.zeros((15000, num_of_stocks))
    ret_arr = np.zeros(15000)
    vol_arr = np.zeros(15000)
    sharpe_arr = np.zeros(15000)
    day_len = 252
    w_transform = weights.T
    retention_mean = log_ret.mean()
 
    for indicator in range(15000):
 
        weights = np.array(np.random.random(18))
        all_weights[indicator,:] = weights / np.sum(weights)
        ret_arr[indicator] = np.sum((retention_mean * weights) * day_len)
        vol_arr[indicator] = np.sqrt(np.dot(w_transform, np.dot(log_ret.cov() * day_len, weights)))
        sharpe_arr[indicator] = ret_arr[indicator]/vol_arr[indicator]
    return weights
w_methodology = weight_methodology(stocks, log_ret, efc)


def get_rvs(w_methodology):
    weights = np.array(w_methodology)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])
 
def negative_sharpe(w_methodology):
    rvs_return = get_rvs(w_methodology)[2]
    neg_rvs = rvs_return * -1
    return neg_rvs
 
def check_sum(w_methodology):
    sum_checker = np.sum(w_methodology) - 1
 
    return sum_checker
 
opt_results = minimize(negative_sharpe, [0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555], method = 'SLSQP', bounds=[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)], constraints=({'type':'eq','fun': check_sum}))
 
 
weights = opt_results.x
 
portfolio_df = pd.DataFrame(tickers)
 
portfolio_df['weights'] = weights
portfolio_df = portfolio_df.rename(columns={0:'ticker'})
output_df = pd.DataFrame(tickers[:18], weights[:18])
print(output_df)

#final_df
tables = jupytab.Tables()


# # Example 1: Static data: it will never change on the Tableau side:
# static_df = dynamic_df()
# tables['static'] = jupytab.DataFrameTable('A static table', dataframe=static_df)

# Example 2: Dynamic data: a new DataFrame is generated whenever Extract is requested on Tableau's side:
tables['portfoliobalanced1'] = jupytab.DataFrameTable('PortfolioBalanced1', dataframe=output_df, include_index=True)

# def multiply(my_first_number, my_second_number):
#     return my_first_number * my_second_number
    
# functions = jupytab.Functions() # Publication-ready functions contained by this notebook


[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
                 0
2.671890e-01  meta
3.660338e-01  adbe
3.868138e-17   amd
5.234701e-02  csco
9.149450e-02  fisv
3.795461e-16   ibm
8.256835e-17   ale
2.428266e-02  lrcx
6.299956e-16  jazz
3.476313e-02    mu
3.241777e-16  orcl
1.649107e-16  qcom
1.020142e-17   txn
2.207295e-16  nvda
2.249614e-17   fis
7.854478e-16   crm
1.005672e-16  eqix
1.638899e-01  vrsk


In [49]:
# GET /schema
tables.render_schema()

[{"id": "portfoliobalanced1", "alias": "PortfolioBalanced1", "columns": [{"id": "index", "dataType": "float"}, {"id": "0", "dataType": "string"}]}]


In [50]:
# GET /data
try:
    tables.render_data(REQUEST)
except NameError:
    print("Not Available outside jupytab context")

Not Available outside jupytab context


In [51]:
# POST /evaluate
try:
    functions.render_evaluate(REQUEST)
except NameError:
    print("Not available outside jupytab context")

Not available outside jupytab context
